In this project, we use Detectron2 to train the Faster-RCNN model on speed limit sign dataset. 

Detectron2 is Facebook AI Research's next generation library that provides state-of-the-art detection and segmentation algorithms. It is the successor of Detectron and maskrcnn-benchmark.

![img](https://github.com/facebookresearch/detectron2/raw/main/.github/Detectron2-Logo-Horz.svg)

# Install detectron2

In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Properly install detectron2. (Please do not install twice in both ways)
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyYAML-5.1-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.0.81 requires PyYAML>=5.3.1, but you have pyyaml 5.1 which is incompatible.
roboflow 1.1.0 requires PyYAML>=5.3.1, but you have pyyaml 5.1 which is incompatible.
pytorch-lightning 1.6.5 requires PyYAML>=5.4, but you have pyyaml 5.1 which is incompatible.
autogluon-vision 0.5.2 requires pandas!=1.4.0,<1.5,>=1.2.5, but you have pandas 1.2.4 which is incompatible.
autogluon-multimodal 0.5.2 requires omegaconf<2.2.0,>=2.1.1, but you have omegaconf 2.2.3 which is incompatible.
autogluon-multimodal 0.5.2 requires pandas!=1.4.0,<1.5,>

In [ ]:
SPEED_THING_CLASSES = ['pl100', 'pl120', 'pl20', 'pl30', 'pl40', 'pl50', 'pl60', 'pl70', 'pl80']
SPEED_THING_DATASET_ID_TO_CONTIGUOUS_ID =  dict(
    ChainMap(*[{i: i} for i in range(9)]))

In [ ]:
import os

# Detectron imports
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances
    
def setup_speed_dataset(dataset_dir):
    train_image_dir = os.path.join(dataset_dir, 'data')
    test_image_dir = os.path.join(dataset_dir, 'data')

    train_json_annotations = os.path.join(
        dataset_dir, 'train_dataset.json')
    test_json_annotations = os.path.join(
        dataset_dir, 'val_dataset.json')

    register_coco_instances(
        "speed_custom_train",
        {},
        train_json_annotations,  
        train_image_dir)
    MetadataCatalog.get(
        "speed_custom_train").thing_classes = SPEED_THING_CLASSES
    MetadataCatalog.get(
        "speed_custom_train").thing_dataset_id_to_contiguous_id = SPEED_THING_DATASET_ID_TO_CONTIGUOUS_ID

    register_coco_instances(
        "speed_custom_val",
        {},
        test_json_annotations,
        test_image_dir)
    MetadataCatalog.get(
        "speed_custom_val").thing_classes = SPEED_THING_CLASSES
    MetadataCatalog.get(
        "speed_custom_val").thing_dataset_id_to_contiguous_id = SPEED_THING_DATASET_ID_TO_CONTIGUOUS_ID

In [ ]:
setup_speed_dataset("/home/hew/tt100k_2021/data/export_dataset")

In [ ]:
import os
import sys


# Detectron imports
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, verify_results


class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        """
        Builds evaluators for post-training mAP report.
        Args:
            cfg(CfgNode): a detectron2 CfgNode
            dataset_name(str): registered dataset name

        Returns:
            detectron2 DatasetEvaluators object
        """
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluators = [COCOEvaluator(dataset_name, cfg, True, output_folder)]
        return DatasetEvaluators(evaluators)

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        """
        Builds DataLoader for test set.
        Args:
            cfg(CfgNode): a detectron2 CfgNode
            dataset_name(str): registered dataset name

        Returns:
            detectron2 DataLoader object specific to the test set.
        """
        return build_detection_test_loader(
            cfg, dataset_name)

    @classmethod
    def build_train_loader(cls, cfg):
        """
        Builds DataLoader for train set.
        Args:
            cfg(CfgNode): a detectron2 CfgNode

        Returns:
            detectron2 DataLoader object specific to the train set.
        """
        return build_detection_train_loader(
            cfg)


def main(args):
    # Setup config node
    cfg = setup_config(args,
                       random_seed=args.random_seed)

    # For debugging only
    #cfg.defrost()
    #cfg.DATALOADER.NUM_WORKERS = 0
    #cfg.SOLVER.IMS_PER_BATCH = 1

    # Eval only mode to produce mAP results
    # Build Trainer from config node. Begin Training.

    trainer = Trainer(cfg)

    trainer.resume_or_load(resume=args.resume)
    return trainer.train()


if __name__ == "__main__":
    # Create arg parser
    arg_parser = setup_arg_parser()

    args = arg_parser.parse_args()
    print("Command Line Args:", args)

    launch(
        main,
        args.num_gpus,
        num_machines=args.num_machines,
        machine_rank=args.machine_rank,
        dist_url=args.dist_url,
        args=(args,),
    )
